## Feature generation

In [1]:
import numpy as np
import pandas as pd
import psycopg2 as pg

In [2]:
df = pd.read_csv("/Users/mahmud/Documents/DSSG-Bootcamp/project-inspector-gadget/DATA/Building_Violations_sample_50000.csv")

/Users/mahmud/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
import re

def spaces_to_snake(column_name):
    '''
    converts a string that has spaces into snake_case
    '''
    
    s = re.sub(r'\s+','_', column_name)
    return s.lower()

df.columns = [spaces_to_snake(col) for col in df.columns]

In [15]:
# Change all dates to datetime format

df.violation_date = pd.to_datetime(df.violation_date)
df.violation_status_date = pd.to_datetime(df.violation_status_date)
df.violation_last_modified_date = pd.to_datetime(df.violation_last_modified_date)



In [34]:
# Creating dummies
dummies = pd.get_dummies(df[['inspection_status', 'inspection_category']])
newdf = pd.merge(df, dummies, left_index = True, right_index = True) 

In [46]:
newdf.dtypes
newdf.head(5)

,unnamed:_0,id,violation_last_modified_date,violation_date,violation_code,violation_status,violation_status_date,violation_description,violation_location,violation_inspector_comments,...,longitude,location,inspection_status_CLOSED,inspection_status_FAILED,inspection_status_HOLD,inspection_status_PASSED,inspection_category_COMPLAINT,inspection_category_PERIODIC,inspection_category_PERMIT,inspection_category_REGISTRATION
0,0,5063843,2015-05-21,2015-05-21,CN065014,OPEN,NaT,REPAIR LINTELS,OTHER : :,LINTELS; RUSTED.,...,-87.606184,"(41.73308939713441,-87.60618433527188)",0,1,0,0,1,0,0,0
1,1,5063990,2015-05-21,2015-05-21,CN044013,OPEN,NaT,PIPE OPENING W/NONCOMBUST MATL,OTHER : :MULTIPLE FLOORS,"OPENINGS AROUND PIPES, SEAL WITH APPROVED MATE...",...,-87.615694,"(41.77708792596553,-87.61569442972947)",0,1,0,0,0,1,0,0
2,2,5064022,2015-05-21,2015-05-21,CN104035,OPEN,NaT,MAINTAIN WINDOW,OTHER : :,"ALL ELEVATIONS , WOOD WINDOW FRAMES AND NORTH...",...,-87.708657,"(41.93799454179893,-87.70865671096144)",0,1,0,0,1,0,0,0
3,3,5064019,2015-05-21,2015-05-21,CN196029,OPEN,NaT,POST OWNER/MANAGERS NAME/#,OTHER : :,BUILDING - NO OWNER'S I.D. SIGN POSTED .,...,-87.708657,"(41.93799454179893,-87.70865671096144)",0,1,0,0,1,0,0,0
4,4,5064139,2015-05-21,2015-05-21,CN104015,OPEN,NaT,"REPLCE WINDOW PANES, PLEXGLAS",OTHER : :,NORTH ELEVATION / 1ST FLOOR - BROKEN WINDOW PANE.,...,-87.620616,"(41.791469839262206,-87.62061589476306)",0,1,0,0,1,0,0,0


In [60]:
# Create a building features dataframe
b_features = pd.DataFrame()
buildings = newdf.groupby('property_group')

In [61]:
b_features['num_failed'] = buildings.inspection_status_FAILED.sum()
b_features['num_passed'] = buildings.inspection_status_PASSED.sum()
b_features['total_inspected'] = b_features.num_failed + b_features.num_passed

b_features['num_complaint'] = buildings.inspection_category_COMPLAINT.sum()

In [62]:
b_features.head(5)

,num_failed,num_passed,total_inspected,num_complaint
property_group,,,,
1001,3,0,3,0
1002,0,2,2,0
1004,1,1,2,2
1007,1,0,1,1
1008,1,0,1,0
